In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, r'C:\Users\lukas\Dokumente\projects\esinet')
import pickle as pkl
import numpy as np
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info

plot_params = dict(surface='white', hemi='both', verbose=0)

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.5s remaining:    2.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.5s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


In [28]:
n_samples = 1
duration_of_trial = 1.0
settings = dict(duration_of_trial=duration_of_trial, target_snr=999999, number_of_sources=1, extents=35)
sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)

Simulate Source


  0%|          | 0/1 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/1 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/1 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


In [31]:
epochs = sim_lstm.eeg_data
epochs.filter(0.1, 45, method='iir').apply_baseline(baseline=(None, None)).set_eeg_reference('average', projection=True)
raw = mne.io.RawArray(np.squeeze(epochs.get_data()), epochs.info)
true_source = sim_lstm.source_data[0]
true_source.data /= np.max(true_source.data)
true_source.plot(**plot_params)

Setting up band-pass filter from 0.1 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 45.00 Hz: -6.02, -6.02 dB

Applying baseline correction (mode: mean)
Creating RawArray with float64 data, n_channels=61, n_times=100
    Range : 0 ... 99 =      0.000 ...     0.990 secs
Ready.


C:\Users\lukas\AppData\Local\Temp/ipykernel_11764/1591048887.py:2: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs.filter(0.1, 45, method='iir').apply_baseline(baseline=(None, None)).set_eeg_reference('average', projection=True)


In [32]:
freq_band_name = 'beta'
freq_band = [13, 30]
freqs = np.logspace(freq_band[0], freq_band[1], num=10)
n_cycles = 2

data_covariance = mne.compute_raw_covariance(raw, verbose=0)
csd = mne.time_frequency.csd_morlet(epochs, freqs, n_cycles=n_cycles, verbose=0).mean()

filters = mne.beamformer.make_dics(epochs.info, fwd, csd, reduce_rank=True, real_filter=True, pick_ori='max-power', verbose=0)


stc, _ = mne.beamformer.apply_dics_csd(csd, filters, verbose=0)

stc.data /= np.max(stc.data)
stc.plot(**plot_params)

C:\Users\lukas\AppData\Local\Temp/ipykernel_11764/1361747469.py:6: RuntimeWarning: Too few samples (required : 310 got : 100), covariance estimate may be unreliable
  data_covariance = mne.compute_raw_covariance(raw, verbose=0)
100%|██████████| CSD epoch blocks : 1/1 [00:00<00:00,   29.42it/s]


Using control points [0.00252016 0.00566715 0.03792423]
Using control points [0.02785156 0.04184901 0.88249387]
Using control points [0.02785156 0.04184901 0.88249387]


In [39]:
noise_cov = mne.compute_raw_covariance(raw, verbose=0)
inverse_operator = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, fixed=True, depth=None, verbose=0)
# stc = mne.minimum_norm.compute_source_psd(raw, inverse_operator, lambda2=1/9, method='eLORETA', tmin=None, tmax=None, fmin=freq_band[0], fmax=freq_band[1])
stc = mne.minimum_norm.source_band_induced_power(epochs, inverse_operator, dict(beta=freq_band), lambda2=1/9, method='eLORETA', verbose=0)

stc.data /= np.abs(stc.data).max()
stc.plot(**plot_params)

C:\Users\lukas\AppData\Local\Temp/ipykernel_11764/4143045682.py:1: RuntimeWarning: Too few samples (required : 310 got : 100), covariance estimate may be unreliable
  noise_cov = mne.compute_raw_covariance(raw, verbose=0)


ValueError: cannot reshape array of size 3852 into shape (1284,3,3)

In [41]:
3852/1284

3.0